<a href="https://colab.research.google.com/github/stefkong1982/netology.ru/blob/Master/Analitika_bolshih_dannyh/Itogovaya_rabota_Analitika_bolshih_dannyh/Kondratev_Itog_Analitika_bolshih_dannyh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Итоговая работа "Аналитика больших данных"

**Преподаватель:** Алексей Кузьмин

В данном разделе мы выступим в роли data scientist и попытаемся построить простую модель для рекомендации фильмов пользователям.

**1. Загрузите в колаб файлы по оценкам (ratings) и фильмам (movies) и создайте на их основе pandas-датафреймы**



Сформировав общий топ фильмов в прошлой практике, мы хотим сделать шаг вперед и начать советовать пользователю те фильмы, которые могли бы быть для него наиболее интересны. Наша цель - научится предсказывать оценку фильма пользователем. Для тестирования модели найдем пользователя, который поставил больше всего оценок

**2. Средствами Pandas, используя dataframe ratings, найдите id пользователя, поставившего больше всего оценок**



Отберем фильмы, которые оценил данный пользователь

**3. Оставьте в датафрейме ratings только те фильмы, который оценил данный пользователь**



Для построения модели нам нужны признаки. В качестве таковых будем использовать:

* Год выхода

* Жанры
* Общее количество оценок

* Суммарную оценку




**4. Добавьте к датафрейму из задания 3 столбцы:**

  * По жанрам. Каждый столбец - это жанр. Единицу записываем, если фильм принадлежит данному жанру и 0 - если нет

  * столбцы с общим количеством оценок от всех пользователей на фильм и суммарной оценкой от всех пользователей



Теперь все готово и можно строить модель


**5. Сформируйте X_train, X_test, y_train, y_test**

**6. Возьмите модель линейной регрессии (или любую другую для задачи регрессии)  и обучите ее на фильмах**

**7. Оцените качество модели на X_test, y_test при помощи метрик для задачи регрессии**


Вторая часть практики Python связана со Spark’ом

**8. Загрузить данные в spark**

**9. Средствами спарка вывести среднюю оценку для каждого фильма**

**10. Посчитайте средствами спарка среднюю оценку для каждого жанра**

**11. В спарке получить 2 датафрейма с 5-ю самыми популярными и самыми непопулярными фильмами (по количеству оценок, либо по самой оценке - на Ваш выбор)**


## Решение

В данном разделе мы выступим в роли data scientist и попытаемся построить простую модель для рекомендации фильмов пользователям.

### 1 - Загрузите в колаб файлы по оценкам (ratings) и фильмам (movies) и создайте на их основе pandas-датафреймы

Сформировав общий топ фильмов в прошлой практике, мы хотим сделать шаг вперед и начать советовать пользователю те фильмы, которые могли бы быть для него наиболее интересны. Наша цель - научится предсказывать оценку фильма пользователем. Для тестирования модели найдем пользователя, который поставил больше всего оценок

In [337]:
import pandas as pd

# Ссылка на файл с оценками
ratings_url = "https://github.com/stefkong1982/netology.ru/raw/Master/Analitika_bolshih_dannyh/Itogovaya_rabota_Analitika_bolshih_dannyh/u.data.csv"

# Загружаем файл с оценками в датафрейм
ratings = pd.read_csv(ratings_url, sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])

# Ссылка на файл с фильмами
movies_url = "https://github.com/stefkong1982/netology.ru/raw/Master/Analitika_bolshih_dannyh/Itogovaya_rabota_Analitika_bolshih_dannyh/u.item.csv"

# Загружаем файл с фильмами в датафрейм
movies = pd.read_csv(movies_url, sep='|', encoding='latin-1', header=None,
                     names=['movie_id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL',
                            'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime',
                            'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery',
                            'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'])

In [338]:
# Посмотрим на первые строки каждого датафрейма
ratings.head(3)

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116


```
u.data — полный набор данных u, 100 000 оценок 943 пользователей по 1682 элементам.
```
```
Каждый пользователь оценил не менее 20 фильмов. Пользователи и элементы
пронумерованы последовательно с 1. Данные случайным образом
заказал.
```

```
Столбцы:

user_id	                          item_id	      rating	      timestamp
идентификатор пользователя | идентификатор элемента | рейтинг | временная метка
```
```
Это список с разделителем табуляция sep='\t'
```
```
Временные метки представляют собой unix-секунды с 01.01.1970 UTC.
```

In [339]:
movies.head(3)

,movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


```
u.item - Информация фильмах:
```

```
Столбцы:

идентификатор фильма | название фильма | дата выпуска | дата выхода видео |
URL-адрес IMDb | неизвестный | Действие | Приключения | Анимация |
Детская | комедия | Преступность | Документальный | Драма | Фэнтези |
Фильм-Нуар | Ужасы | Музыкальный | Тайна | Романтика | Научная фантастика |
Триллер | Война | Западная |
```           
```
Это список с разделителем sep='|'
```
```
Последние 19 полей — жанры,

1 — фильм. относится к этому жанру

0 - означает, что это не так

фильмы могут быть внесколько жанров одновременно
```
```
Идентификаторы фильмов (movie_id) — это те, которые используются в наборе данных u.data.
```

### 2 - Средствами Pandas, используя dataframe `ratings`, найдите id пользователя, поставившего больше всего оценок

In [340]:
# Группируем данные по 'user_id' и считаем количество оценок каждого пользователя
# По умолчанию метод `value_counts()` в Pandas сортирует уникальные значения в порядке убывания количества их появлений.
user_ratings_count = ratings['user_id'].value_counts()

# Определяем top_user_id - ID пользователя с наибольшим количеством оценок
top_user_id = user_ratings_count.index[0]

print(f"ID пользователя с наибольшим количеством оценок: {top_user_id}")
print(f"Количество оценок пользователя: {user_ratings_count.values[0]}")


ID пользователя с наибольшим количеством оценок: 405
Количество оценок пользователя: 737


Отберем фильмы, которые оценил данный пользователь

### 3 - Оставьте в датафрейме ratings только те фильмы, который оценил данный пользователь

Для построения модели нам нужны признаки. В качестве таковых будем использовать:
* Год выхода
* Жанры
* Общее количество оценок
* Суммарную оценку


In [341]:
# Вычисляем общее количество оценок и суммарную оценку от всех пользователей для каждого фильма
ratings['total_ratings_all'] = ratings.groupby('movie_id')['rating'].transform('count')
ratings['sum_ratings_all'] = ratings.groupby('movie_id')['rating'].transform('sum')

# Отображаем обновленный датафрейм ratings
ratings

,user_id,movie_id,rating,timestamp,total_ratings_all,sum_ratings_all
0,196,242,3,881250949,117,467
1,186,302,3,891717742,297,1236
2,22,377,1,878887116,13,28
3,244,51,2,880606923,81,280
4,166,346,1,886397596,126,459
...,...,...,...,...,...,...
99995,880,476,3,880175444,160,483
99996,716,204,5,879795543,350,1342
99997,276,1090,1,874795795,37,89
99998,13,225,2,882399156,109,317


In [342]:
# Оставляем только фильмы, оцененные пользователем - 405 (top_user_id)
user_ratings = ratings[ratings['user_id'] == top_user_id]
user_ratings.head(3)

,user_id,movie_id,rating,timestamp,total_ratings_all,sum_ratings_all
12276,405,56,4,885544911,394,1600
12383,405,592,1,885548670,9,30
12430,405,1582,1,885548670,1,1


### 4 - Добавьте к датафрейму из задания 3 столбцы:

* Жанр
* Год выхода
* столбец с общим количеством оценок от всех пользователей на фильм - уже добавили на в 3-м задании
* столбец и суммарной оценкой от всех пользователей - уже добавили на в 3-м задании


In [343]:
# Добавляем к датафрейму столбцы 'жанр', 'год выхода'
user_ratings = user_ratings.merge(movies[['movie_id', 'release_date'] + list(movies.columns[5:24])], on='movie_id', how='left')
user_ratings.head(1)

,user_id,movie_id,rating,timestamp,total_ratings_all,sum_ratings_all,release_date,unknown,Action,Adventure,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,405,56,4,885544911,394,1600,01-Jan-1994,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [344]:
# Преобразование столбца с датой
user_ratings['release_date'] = pd.to_datetime(user_ratings['release_date'])
user_ratings.head(1)

,user_id,movie_id,rating,timestamp,total_ratings_all,sum_ratings_all,release_date,unknown,Action,Adventure,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,405,56,4,885544911,394,1600,1994-01-01,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [345]:
# Извлечение года из даты и замена этим значением столбца release_date
user_ratings['release_date'] = user_ratings['release_date'].dt.year
user_ratings.head(1)

,user_id,movie_id,rating,timestamp,total_ratings_all,sum_ratings_all,release_date,unknown,Action,Adventure,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,405,56,4,885544911,394,1600,1994,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Теперь все готово и можно строить модель

### 5 - Сформируйте X_train, X_test, y_train, y_test

In [354]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score, mean_absolute_percentage_error, mean_squared_error

In [355]:
# Выделение признаков (X) и целевой переменной (y)
X = user_ratings[list(user_ratings.columns[4:26])]
y = user_ratings['rating']

In [356]:
X.head(1)

,total_ratings_all,sum_ratings_all,release_date,unknown,Action,Adventure,Animation,Children,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,394,1600,1994,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [357]:
# Разбиение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 6 - Возьмите модель линейной регрессии (или любую другую для задачи регрессии)  и обучите ее на фильмах

In [358]:
# Создание модели линейной регрессии
model = LinearRegression()

# Обучение модели на данных
model.fit(X_train, y_train)

LinearRegression()

### 7 - Оцените качество модели на X_test, y_test при помощи метрик для задачи регрессии

In [363]:
# Предсказание на тестовом наборе
y_pred = model.predict(X_test)

In [364]:
# DataFrame для удобства сравнения
predictions = pd.DataFrame({'Фактическое значение': y_test, 'Предсказанное значение': y_pred})
predictions.head()

,Фактическое значение,Предсказанное значение
669,1,1.136713
33,1,1.775713
549,1,1.083887
199,1,1.341517
264,1,3.029876


In [365]:
# Вычисление точности модели с использованием различных метрик
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)  # RMSE - корень из MSE

print(f'Mean Absolute Error (MAE): {mae:.2f}')
print(f'R^2 Score: {r2:.2f}')
print(f'Mean Absolute Percentage Error (MAPE): {mape:.2f}')
print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')

Mean Absolute Error (MAE): 0.95
R^2 Score: 0.16
Mean Absolute Percentage Error (MAPE): 0.55
Root Mean Squared Error (RMSE): 1.30


Значения метрик ошибок (MAE, R^2, MAPE, RMSE) в контексте задачи предсказания рейтинга.

1. Mean Absolute Error (MAE) равный 0.95:
   - MAE измеряет среднюю абсолютную ошибку модели. Значение 0.95 означает, что в среднем модель ошибается на примерно 0.95 балла в оценке рейтинга. Чем ниже MAE, тем лучше.

2. R^2 Score равный 0.16:
   - R^2 Score или коэффициент детерминации измеряет объясненную вариацию моделью. Значение 0.16 говорит о том, что модель объясняет примерно 16% изменчивости целевой переменной. Чем ближе значение R^2 к 1, тем лучше модель объясняет данные.

3. Mean Absolute Percentage Error (MAPE) равный 0.55:
   - MAPE измеряет среднюю абсолютную процентную ошибку модели. Значение 0.55 означает, что модель в среднем ошибается на 55% от фактического значения рейтинга. Чем ниже MAPE, тем лучше.

4. Root Mean Squared Error (RMSE) равный 1.30:
   - RMSE измеряет среднеквадратичную ошибку модели. Значение 1.30 означает, что корень из среднеквадратичной ошибки составляет приблизительно 1.30 балла. Чем ниже RMSE, тем лучше модель предсказывает.

Исходя из значений метрик, модель, вероятно, не слишком точно предсказывает рейтинги и есть место для улучшения. Будет полезно анализировать причины ошибок и возможно рассмотреть использование других признаков или моделей для повышения точности предсказаний.

Вторая часть практики Python связана со Spark’ом




In [368]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=a1ad87f5bed60ed2a49100f988572414723b84cd3c022bc2312c8de696447905
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


### 8 - Загрузить данные в spark

In [377]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

# Создаем сессию Spark
spark = SparkSession.builder.appName("PandasToSpark").getOrCreate()
sqlContext = SQLContext(spark)

# Преобразуем Pandas DataFrame в Spark DataFrame
ratings_spark = sqlContext.createDataFrame(ratings)
movies_spark = sqlContext.createDataFrame(movies)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [378]:
# Отображение первых 5 строк Spark DataFrame
ratings_spark.show(5)

+-------+--------+------+---------+-----------------+---------------+
|user_id|movie_id|rating|timestamp|total_ratings_all|sum_ratings_all|
+-------+--------+------+---------+-----------------+---------------+
|    196|     242|     3|881250949|              117|            467|
|    186|     302|     3|891717742|              297|           1236|
|     22|     377|     1|878887116|               13|             28|
|    244|      51|     2|880606923|               81|            280|
|    166|     346|     1|886397596|              126|            459|
+-------+--------+------+---------+-----------------+---------------+
only showing top 5 rows



In [379]:
movies_spark.show(5)

+--------+-----------------+------------+------------------+--------------------+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|movie_id|      movie_title|release_date|video_release_date|            IMDb_URL|unknown|Action|Adventure|Animation|Children|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+--------+-----------------+------------+------------------+--------------------+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|       1| Toy Story (1995)| 01-Jan-1995|               NaN|http://us.imdb.co...|      0|     0|        0|        1|       1|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|
|       2| GoldenEye (1995)| 01-Jan-1995|               NaN|http

### 9 - Средствами спарка вывести среднюю оценку для каждого фильма

In [385]:
from pyspark.sql.functions import avg

# Рассчитываем среднюю оценку для каждого фильма
average_ratings = ratings_spark.groupBy("movie_id").agg(avg("rating").alias("average_rating"))
average_ratings.show(5)

+--------+------------------+
|movie_id|    average_rating|
+--------+------------------+
|     474| 4.252577319587629|
|      29|2.6666666666666665|
|      26| 3.452054794520548|
|     964|3.3333333333333335|
|      65|3.5391304347826087|
+--------+------------------+
only showing top 5 rows



In [388]:
# Присоединяем информацию о фильмах к таблице средних оценок
result = average_ratings.join(movies_spark, "movie_id")
result.show(5)

+--------+------------------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|movie_id|    average_rating|         movie_title|release_date|video_release_date|            IMDb_URL|unknown|Action|Adventure|Animation|Children|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+--------+------------------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|      26| 3.452054794520548|Brothers McMullen...| 01-Jan-1995|               NaN|http://us.imdb.co...|      0|     0|        0|        0|       0|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     

In [390]:
# Показываем результат
result.select("movie_id", "movie_title", "average_rating").show()

+--------+--------------------+------------------+
|movie_id|         movie_title|    average_rating|
+--------+--------------------+------------------+
|      26|Brothers McMullen...| 3.452054794520548|
|      29|Batman Forever (1...|2.6666666666666665|
|     474|Dr. Strangelove o...| 4.252577319587629|
|      65|What's Eating Gil...|3.5391304347826087|
|     191|      Amadeus (1984)| 4.163043478260869|
|     418|   Cinderella (1950)|3.5813953488372094|
|     541|Mortal Kombat (1995)| 2.877551020408163|
|     558|Heavenly Creature...|3.6714285714285713|
|     222|Star Trek: First ...|  3.66027397260274|
|     270|      Gattaca (1997)|3.5955882352941178|
|     293|Donnie Brasco (1997)| 3.802721088435374|
|     730|Queen Margot (Rei...|               3.5|
|     243|Jungle2Jungle (1997)|2.4393939393939394|
|     278| Bed of Roses (1996)|3.2333333333333334|
|     367|     Clueless (1995)| 3.458823529411765|
|     442|Amityville Curse,...|              1.25|
|     705|Singin' in the Ra...|

### 10 - Посчитайте средствами спарка среднюю оценку для каждого жанра

In [400]:
# Объединяем таблицы рейтингов и фильмов по movie_id
merged_data = ratings_spark.join(movies_spark, "movie_id")
merged_data.show(3)

+--------+-------+------+---------+-----------------+---------------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|movie_id|user_id|rating|timestamp|total_ratings_all|sum_ratings_all|         movie_title|release_date|video_release_date|            IMDb_URL|unknown|Action|Adventure|Animation|Children|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+--------+-------+------+---------+-----------------+---------------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|      26|    686|     5|879546847|               73|            252|Brothers McMullen...| 01-Jan-1995|         

In [410]:
from pyspark.sql.functions import col

# Фильтруем строки, где unknown не равно 0, затем агрегируем данные по рейтингу
unknown_ratings_per_genre = merged_data.filter(col("unknown") != 0).groupBy("unknown").agg(avg("rating").alias("unknown_ratings_per_genre"))

unknown_ratings_per_genre.show(3)


+-------+-------------------------+
|unknown|unknown_ratings_per_genre|
+-------+-------------------------+
|      1|                      3.2|
+-------+-------------------------+



In [415]:
from pyspark.sql.functions import col, avg
from functools import reduce

# Фильтруем строки, где значение жанра не равно 0, затем агрегируем данные по рейтингу для всех жанров
genre_columns = ['unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy',
                 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
                 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

# Подготавливаем условие для фильтрации всех столбцов с жанрами, где значение не равно 0
conditions = [(col(genre) != 0) for genre in genre_columns]
# Фильтруем строки, где значение жанра не равно 0, затем агрегируем данные по рейтингу для всех жанров
ratings_per_genre = merged_data.filter(reduce(lambda x, y: x | y, conditions)).groupBy(genre_columns).agg(avg("rating").alias("average_rating"))
ratings_per_genre.show()


+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+------------------+
|unknown|Action|Adventure|Animation|Children|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|    average_rating|
+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+------------------+
|      0|     1|        0|        0|       0|     0|    0|          0|    0|      0|        0|     0|      0|      1|      1|     0|       1|  0|      0|3.4237288135593222|
|      0|     0|        0|        0|       0|     0|    0|          0|    0|      0|        0|     1|      0|      0|      1|     0|       1|  0|      0| 4.100418410041841|
|      0|     1|        1|        0|       0|     0|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|   

In [417]:
from pyspark.sql.functions import when, concat_ws
# Фильтруем строки, где значение жанра не равно 0, затем агрегируем данные по рейтингу для всех жанров
genre_columns = ['unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy',
                 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
                 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

# Подготавливаем условие для фильтрации всех столбцов с жанрами, где значение не равно 0
conditions = [(col(genre) != 0) for genre in genre_columns]

# Фильтруем строки, где значение жанра не равно 0, затем агрегируем данные по рейтингу для всех жанров
ratings_per_genre = merged_data.filter(reduce(lambda x, y: x | y, conditions)).groupBy(genre_columns).agg(avg("rating").alias("average_rating"))

# Преобразуем результат в новый формат для вывода
ratings_per_genre_formatted = ratings_per_genre.select(concat_ws(" | ", *[when(col(genre) != 0, genre).otherwise("").alias("жанры") for genre in genre_columns]).alias("жанры"), col("average_rating"))

ratings_per_genre_formatted.show(truncate=False)


+-------------------------------------------------------------------------------------------------+------------------+
|жанры                                                                                            |average_rating    |
+-------------------------------------------------------------------------------------------------+------------------+
| | Action |  |  |  |  |  |  |  |  |  |  |  | Mystery | Romance |  | Thriller |  |                |3.4237288135593222|
| |  |  |  |  |  |  |  |  |  |  | Horror |  |  | Romance |  | Thriller |  |                       |4.100418410041841 |
| | Action | Adventure |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  | Western                     |3.2083333333333335|
| |  |  |  |  |  |  |  |  |  |  |  |  |  | Romance |  | Thriller |  |                             |3.566666666666667 |
| |  |  |  |  |  |  |  | Drama |  |  |  | Musical |  |  |  |  | War |                             |3.3508771929824563|
| | Action |  |  |  |  | Crime |  |  |  |  |  | 

In [407]:
unknown_ratings_per_genre.show(3)

+-------+-------------------------+
|unknown|unknown_ratings_per_genre|
+-------+-------------------------+
|      0|       3.5298929892989297|
|      1|                      3.2|
+-------+-------------------------+







11. В спарке получить 2 датафрейма с 5-ю самыми популярными и самыми непопулярными фильмами (по количеству оценок, либо по самой оценке - на Ваш выбор)
